In [1]:
import cv2
import pandas as pd
import os
from datetime import datetime
import csv
import time
import face_recognition
from scipy.spatial import distance
import dlib
from imutils import face_utils
import torch
import torchvision.transforms as transforms
import joblib
from efficientnet_pytorch import EfficientNet
from PIL import Image

In [2]:
# Load face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

# Indices of facial landmarks for left and right eyes
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [3]:
def save_face_counts(timestamp, face_count, csv_path):
    with open(csv_path, "a", newline="\n") as file:
        writer = csv.writer(file)
        if file.tell() == 0:
            writer.writerow(['time', 'face_count'])
        writer.writerow([timestamp, face_count])

In [4]:
def save_ear(timestamp, ear_value, istired, csv_path):
    with open(csv_path, "a", newline="\n") as file:
        writer = csv.writer(file)
        if file.tell() == 0:
            writer.writerow(['time', 'ear_value', 'istired'])
        writer.writerow([timestamp, ear_value, istired])

In [5]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [6]:
def calculate_tiredness(frame, timestamp):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detector(gray, 0)

    for subject in subjects:
        shape = predictor(gray, subject)
        shape = face_utils.shape_to_np(shape)

        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        if ear < 0.25:
            print("Tiredness detected")
            save_ear(timestamp, ear, istired=1, csv_path=r"output\ear_values.csv")
        else:
            print("No tiredness detected")
            save_ear(timestamp, ear, istired=0, csv_path=r"output\ear_values.csv")

In [ ]:
categories = ['writing_reading', 'distracted_mouth_open', 'using_smartphone', 'focused_mouth_closed', 
              'distracted_mouth_closed', 'fatigue', 'focused_mouth_open', 'raise_hand', 'listening', 'sleeping']

In [1]:
label_map = {'writing_reading': 0,
 'distracted_mouth_open': 1,
 'using_smartphone': 2,
 'focused_mouth_closed': 3,
 'distracted_mouth_closed': 4,
 'fatigue': 5,
 'focused_mouth_open': 6,
 'raise_hand': 7,
 'listening': 8,
 'sleeping': 9}

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
torch.cuda.is_available()

True

In [11]:
# Later, to load the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EfficientNet.from_pretrained('efficientnet-b0')
model.load_state_dict(torch.load('models\efficientnet_b0.pth'))
model.to(device)
model.eval()

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [12]:
# load the machine learning algorithm
classifier = joblib.load(r"models\Logistic Regression_model.pkl")

c:\Users\adila\anaconda3\envs\ml-env\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
# Function to preprocess image
def preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img_tensor = preprocess(img).unsqueeze(0).to(device)
    return img_tensor

In [14]:
# Function to test new images
def test_new_images(new_image_paths, classifier, label_map):
    for image_path in new_image_paths:
        img_tensor = preprocess_image(image_path)
        with torch.no_grad():
            feature = model.extract_features(img_tensor).flatten().cpu().numpy()
        prediction = classifier.predict([feature])
        predicted_label = list(label_map.keys())[list(label_map.values()).index(prediction[0])]
        print(f"Image: {image_path} -> Predicted Label: {predicted_label}")

In [17]:
new_image_paths = [
    r'C:\Users\adila\OneDrive\Images\profile.png',

]

test_new_images(new_image_paths, classifier, label_map)

Image: C:\Users\adila\OneDrive\Images\profile.png -> Predicted Label: sleeping


In [9]:
def process_video(width, height, video_path, csv_path):
    cap = cv2.VideoCapture(video_path)
    last_frame_time = time.time()

    while True:
        success, frame = cap.read()
        if not success:
            break
        else:
            timestamp = datetime.now().strftime("%M:%S:%f")
            # Resize the frame
            frame = cv2.resize(frame, (width, height))
            
            # Convert the frame from BGR (OpenCV's default) to RGB (face_recognition's requirement)
            rgb_frame = frame[:, :, ::-1]

            # Use face_recognition to detect faces
            face_locations = face_recognition.face_locations(rgb_frame, model="hog")

            # Draw bounding box around faces
            for (top, right, bottom, left) in face_locations:
                cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)

            # Save the number of faces detected in the current frame
            face_count = len(face_locations)
            
            # Write the face count to the CSV file
            save_face_counts(timestamp, face_count, csv_path)
            
            # # Calculate tiredness every second
            # current_time = time.time()
            # if current_time - last_frame_time >= 1:
            #     calculate_tiredness(rgb_frame, timestamp)
            #     last_frame_time = current_time
            
            calculate_tiredness(rgb_frame, timestamp)
            
            # Display the frame
            cv2.imshow("Video Stream", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage
width = 640
height = 480
video_path = r"src\vid1.mp4"
csv_path = r"output\face_counts.csv"

process_video(width, height, video_path, csv_path)

Tiredness detected
No tiredness detected
Tiredness detected


In [ ]:
engagement_count = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ]



In [1]:
hehe = "focused_mouth_closed"

label = ' '.join([word.capitalize() for word in hehe.split("_")])
label

'Focused Mouth Closed'